In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint
from datetime import datetime, timedelta, timezone
import pytz
import scipy.stats
from dateutil.relativedelta import relativedelta

import warnings

# Suppress the specific warnings
warnings.filterwarnings("ignore")

today = datetime.utcnow()

from api_keys import espn_link

In [2]:
homeOnlySked = pd.read_csv('data/sked.csv', index_col=False)
homeOnlySked

completeSked = pd.read_csv('data/sked_full.csv', index_col=False)
completeSked

team_names = pd.read_csv('data/team_names.csv', index_col=False)
team_names

file_name = f"data/allG_df_fp-{(today-timedelta(days=1)).strftime('%Y-%m-%d')}.csv"
allG_df = pd.read_csv(file_name, index_col=False)
allG_df

file_name = f"data/all_df_fp-{(today-timedelta(days=1)).strftime('%Y-%m-%d')}.csv"
all_df = pd.read_csv(file_name, index_col=False)
all_df

team_names_only = team_names[['abbreviation', 'name']]

all_df_forwards =  all_df.loc[all_df['position'] != 'D']
all_df_defense =  all_df.loc[all_df['position'] == 'D']

all_df_centers =  all_df.loc[all_df['secondaryPosition'] == 'C']
all_df_wingers =  all_df.loc[all_df['secondaryPosition'] == 'W']

file_name = f"data/summary_stats-{(today-timedelta(days=1)).strftime('%Y-%m-%d')}.csv"
player_bios = pd.read_csv(file_name, index_col=False)

# file_name = f"data/rankings_worksheet_{(today-timedelta(days=6)).strftime('%Y-%m-%d')}.xlsx"
# rankings = pd.read_excel(file_name) +timedelta(days=1)

In [3]:
def summary_statistics(df):
    temp_build = df.groupby('playerId').agg({
        'name': 'first',
        'team': 'last',
        'position': 'first',
        'toi': 'sum',
        'gamesPlayed': 'sum',
        'goals': 'sum',
        'assists': 'sum',
        'specialTeams': 'sum',
        'shots': 'sum',
        'hits': 'sum',
        'blockedShots': 'sum',
        'powerPlayToi': 'sum',
        'shorthandedToi': 'sum',
        'plusMinus': 'sum',
        'fantasyPoints': 'sum',
        'secondaryPosition': 'first',
        'tertiaryPosition': 'first'
    }).reset_index()



    temp_build['FPP60'] = (temp_build['fantasyPoints'] / temp_build['toi'] * 3600).round(2)
    temp_build['FPPG'] = (temp_build['fantasyPoints'] / temp_build['gamesPlayed']).round(2)

    temp_build = temp_build.sort_values(by='FPPG', ascending=False)
    
    return temp_build

In [4]:
player_bios['projectedFantasy'] = player_bios['fantasyPoints'] + player_bios['fantasyPointsRemain']
now = player_bios.loc[(player_bios['position'] == 'D') & (player_bios['gamesPlayed'] >= 5)].sort_values('projectedFantasy', ascending=False)
now['rank'] = now['fantasyPoints'].rank(ascending=False, method='first')
now['projRank'] = now['projectedFantasy'].rank(ascending=False, method='first')
now

,playerId,name,team,position,toi,gamesPlayed,fantasyPoints,FPP60,FPPG,missedGames,...,birthDate,shootsCatches,height,weight,roster_percent,default_pos,espnId,projectedFantasy,rank,projRank
4,8480069,C. Makar,COL,D,33757,23,75.9,8.09,3.30,0,...,1998-10-30,R,71,187,99.906106,D,4233563,267.30,1.0,1.0
7,8480865,N. Dobson,NYI,D,34704,23,68.7,7.13,2.99,0,...,2000-01-07,R,76,200,95.598729,D,4352732,245.11,2.0,2.0
14,8476885,J. Trouba,NYR,D,31471,23,63.5,7.26,2.76,0,...,1994-02-26,R,75,209,97.629640,D,2976839,226.34,4.0,3.0
20,8480800,Q. Hughes,VAN,D,37174,25,68.4,6.62,2.74,0,...,1999-10-14,L,70,180,99.036503,D,4320548,224.58,3.0,4.0
26,8481542,M. Seider,DET,D,31134,23,59.8,6.91,2.60,0,...,2001-04-06,R,75,205,99.391882,D,4565227,213.20,7.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
732,8481564,R. Johnson,BUF,D,11023,13,6.9,2.25,0.53,0,...,2001-07-24,L,72,170,0.069334,D,0,37.11,227.0,240.0
709,8477073,K. MacDermid,COL,D,1292,5,2.9,8.08,0.58,0,...,1994-03-25,L,77,233,0.108339,D,2992930,36.54,243.0,241.0
759,8478854,R. Shea,PIT,D,13965,19,7.6,1.96,0.40,0,...,1997-02-11,L,74,200,0.043334,D,3942033,31.20,225.0,242.0
746,8477810,J. Hanley,DAL,D,3691,5,2.3,2.24,0.46,0,...,1991-06-08,L,71,186,0.057781,D,3149612,29.90,244.0,243.0


In [5]:
trim = all_df.loc[all_df['gameDate'] <= '2023-10-24']
trim = trim.loc[trim['position'] == 'D']
oct24 = summary_statistics(trim).sort_values('fantasyPoints', ascending=False).head(25)
oct24['rank'] = oct24['fantasyPoints'].rank(ascending=False, method='first')

print('<p>')
for i, r in oct24.iterrows():
    print(
        f"[b]{player_bios.loc[player_bios['playerId'] == r['playerId']]['fullName'].iloc[0]}, D, "
        f"{team_names_only.loc[team_names_only['abbreviation'] == r['team']]['name'].iloc[0]} "
        f"(then: {int(r['rank'])}, now: {int(now.loc[now['playerId'] == r['playerId']]['rank'].iloc[0])}, "
        f"later: {int(now.loc[now['playerId'] == r['playerId']]['projRank'].iloc[0])}) <br />"
    )
print('</p>')

<p>
[b]Cale Makar, D, Colorado Avalanche (then: 1, now: 1, later: 1) <br />
[b]Moritz Seider, D, Detroit Red Wings (then: 2, now: 7, later: 5) <br />
[b]Jaccob Slavin, D, Carolina Hurricanes (then: 3, now: 47, later: 45) <br />
[b]Shayne Gostisbehere, D, Detroit Red Wings (then: 4, now: 46, later: 40) <br />
[b]Evan Bouchard, D, Edmonton Oilers (then: 5, now: 10, later: 8) <br />
[b]Noah Dobson, D, New York Islanders (then: 6, now: 2, later: 2) <br />
[b]Rasmus Dahlin, D, Buffalo Sabres (then: 7, now: 5, later: 10) <br />
[b]Jacob Trouba, D, New York Rangers (then: 8, now: 4, later: 3) <br />
[b]Shea Theodore, D, Vegas Golden Knights (then: 9, now: 32, later: 38) <br />
[b]Victor Hedman, D, Tampa Bay Lightning (then: 10, now: 14, later: 22) <br />
[b]Jake Sanderson, D, Ottawa Senators (then: 11, now: 50, later: 19) <br />
[b]Alexandre Carrier, D, Nashville Predators (then: 12, now: 62, later: 83) <br />
[b]Ivan Provorov, D, Columbus Blue Jackets (then: 13, now: 17, later: 33) <br />
[b

In [6]:
now25 = now.head(50)
now25

print('<p>')
for i, r in now25.iterrows():
    print(
        f"[b]{int(r['projRank'])}. {player_bios.loc[player_bios['playerId'] == r['playerId']]['fullName'].iloc[0]}, D, "
        f"{team_names_only.loc[team_names_only['abbreviation'] == r['team']]['name'].iloc[0]} "
        f"(blocks rank: {int(stats.loc[stats['playerId'] == r['playerId']]['bsRank'].iloc[0])}; "
        f"PP time rank: {int(stats.loc[stats['playerId'] == r['playerId']]['ppRank'].iloc[0])}) <br />"
    )
print('</p>')

<p>


NameError: name 'stats' is not defined

In [ ]:
stats = summary_statistics(all_df).sort_values('blockedShots', ascending=False)
stats = stats.loc[stats['position'] == 'D']
stats['bsRank'] = stats['blockedShots'].rank(ascending=False)
stats['ppRank'] = stats['powerPlayToi'].rank(ascending=False)

stats

In [9]:
# player_bios['projectedFantasy'] = player_bios['fantasyPoints'] + player_bios['fantasyPointsRemain']
# now = player_bios.loc[(player_bios['position'] == 'D') & (player_bios['gamesPlayed'] >= 5)].sort_values('projectedFantasy', ascending=False)
# now['rank'] = now['fantasyPoints'].rank(ascending=False, method='first')
# now['projRank'] = now['projectedFantasy'].rank(ascending=False, method='first')
# now +timedelta(days=1) +timedelta(days=1)

fileName = f"data/rankings-final-Dec06.xlsx"
rankings_file = pd.read_excel(fileName)
rankings_file

rankings_file['projectedFantasy'] = rankings_file['fantasyPoints'] + rankings_file['adjSkedFantasyRemain']
rankings_file
rankings_file['projRank'] = rankings_file['projectedFantasy'].rank(ascending=False, method='first')
rankings_file.sort_values('projRank', ascending=True)

rankings_file = rankings_file.loc[rankings_file['gamesPlayed'] >= 5]

In [20]:
for index, row in east.iterrows():
    temp = rankings_file.loc[rankings_file['team'] == row['abbreviation']]
    temp = temp.sort_values('projectedFantasy', ascending=False)
    print(f"{row['name']}")
    print(
        f"<p><b>{player_bios.loc[player_bios['playerId'] == temp['playerId'].iloc[0]]['fullName'].iloc[0]}"
        f", {player_bios.loc[player_bios['playerId'] == temp['playerId'].iloc[0]]['default_pos'].iloc[0]}"
        f"</b>: projected fantasy points: {round(temp['projectedFantasy'].iloc[0], 2)}"
        f"; projected overall rank: {int(temp['projRank'].iloc[0])})</p>")
    print()    
    for k in range(4):
        p = k+1
        print(
            f"{player_bios.loc[player_bios['playerId'] == temp['playerId'].iloc[p]]['fullName'].iloc[0]}"
            f", {player_bios.loc[player_bios['playerId'] == temp['playerId'].iloc[p]]['default_pos'].iloc[0]}"
            f": (projected FP: {round(temp['projectedFantasy'].iloc[p], 2)}"
            f"; projected rank: {int(temp['projRank'].iloc[p])})")
    print()

New York Rangers
<p><b>Jacob Trouba, D</b>: projected fantasy points: 227.19; projected overall rank: 13)</p>

Artemi Panarin, LW: (projected FP: 216.53; projected rank: 18)
Igor Shesterkin, G: (projected FP: 195.12; projected rank: 45)
Chris Kreider, LW: (projected FP: 186.81; projected rank: 54)
Vincent Trocheck, C: (projected FP: 185.66; projected rank: 57)

Boston Bruins
<p><b>David Pastrnak, RW</b>: projected fantasy points: 257.36; projected overall rank: 4)</p>

Charlie McAvoy, D: (projected FP: 196.51; projected rank: 43)
Jeremy Swayman, G: (projected FP: 195.65; projected rank: 44)
Brad Marchand, LW: (projected FP: 175.85; projected rank: 78)
Charlie Coyle, C: (projected FP: 160.46; projected rank: 114)

Florida Panthers
<p><b>Sam Reinhart, RW</b>: projected fantasy points: 231.99; projected overall rank: 12)</p>

Sergei Bobrovsky, G: (projected FP: 177.34; projected rank: 73)
Aleksander Barkov, C: (projected FP: 170.55; projected rank: 92)
Carter Verhaeghe, LW: (projected FP:

In [19]:
team_names_only

teamURL = 'https://api-web.nhle.com/v1/standings/now'

teams = requests.get(teamURL).json()
teams = teams['standings']

team_names = pd.DataFrame()

for team in teams:
    abbreviation = team['teamAbbrev']['default']
    name = team['teamName']['default']
    logoURL = team['teamLogo']
    gamesPlayed = team['gamesPlayed']
    gamesRemaining = 82 - gamesPlayed
    goalsFor = team['goalFor']
    goalsAgainst = team['goalAgainst']
    conference = team['conferenceName']
    division = team['divisionName']

    temp = pd.DataFrame({'abbreviation': [abbreviation], 'name': [name], 'gamesPlayed': [gamesPlayed],
                         'gamesRemaining': [gamesRemaining], 'goalsFor': [goalsFor], 'goalsAgainst': [goalsAgainst], 'logo': [logoURL],
                        'conference': [conference], 'division': [division]})

    team_names = pd.concat([team_names, temp], ignore_index=True)

    
teamList = team_names['abbreviation'].tolist()
team_names

team_names.to_csv('data/team_names.csv', index=False)
west = team_names.loc[team_names['conference'] == 'Western']
east = team_names.loc[team_names['conference'] == 'Eastern']

In [74]:
player_bios.columns.to_list()

['playerId',
 'name',
 'team',
 'position',
 'toi',
 'gamesPlayed',
 'fantasyPoints',
 'FPP60',
 'FPPG',
 'missedGames',
 'gamesRemaining',
 'fantasyPointsRemain',
 'fullName',
 'birthDate',
 'shootsCatches',
 'height',
 'weight',
 'roster_percent',
 'default_pos',
 'espnId',
 'projectedFantasy']